# Planning report
## Create new tickets with fitting predictions easily

In [1]:
import ipywidgets as widgets
import pandas as pd
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)


import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

from datetime import date
from ipywidgets import interact, interact_manual
from src.db.utils import SnowflakeWrapper
from src.compute.utils import Interval
from src.compute.tickets import get_tickets
conn = SnowflakeWrapper.create_snowflake_connection()
sw = SnowflakeWrapper(conn)

In [2]:
#widgets.DatePicker(description='Pick a Date to', disabled=False)
interval = Interval(date(2019, 10, 1), date(2020, 1, 1))
df = get_tickets(sw, interval)

In [3]:
def plot_up_to(x_col, y_col, date_from, date_to):
    interval = Interval(date_from, date_to)
    plot_df = get_tickets(sw, interval)
    return px.histogram(
        plot_df,
        x=x_col,
        color=y_col,
        title=f"Issues on thee interval {interval})",
        hover_data=plot_df.columns,
        labels = {
            # "IssueType": "Issue type",
            # "IssuePriority": "Issue priority",
            "count": "Count of tickets"
        }
    )

_ = interact(
        plot_up_to,
        x_col=widgets.Dropdown(options=["MERGED_STATUS", "IssueType", "IssuePriority"]),
        y_col=widgets.Dropdown(options=["MERGED_STATUS", "IssueType", "IssuePriority"]),
        date_from = widgets.DatePicker(value=pd.to_datetime('2019-10-01')),
        date_to = widgets.DatePicker(value=pd.to_datetime('2020-01-01'))
    )

interactive(children=(Dropdown(description='x_col', options=('MERGED_STATUS', 'IssueType', 'IssuePriority'), v…